In [1]:
import math
import torch.nn as nn
import torch
from matplotlib import pyplot as plt
import torch.optim as optim

In [2]:
math.tanh(-2.2)

-0.9757431300314515

In [3]:
math.tanh(0.1)

0.09966799462495582

In [4]:
math.tanh(2.5)
t_c = [0.5, 14, 15, 28, 11, 8, 3, -4, 6, 13, 21]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

n_sample = t_u.shape[0]
n_val = int(0.2 * n_sample)
shuffled_indices = torch.randperm(n_sample)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
train_indices, val_indices 

train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]
train_t_un = 0.1*train_t_u
val_t_un = 0.1*val_t_u

In [5]:
linear_model = nn.Linear(1, 1)


In [6]:
x = torch.ones(1)

In [7]:
linear_model(x)

tensor([-0.2629], grad_fn=<AddBackward0>)

In [8]:
t_c = [0.5, 14, 15, 28, 11, 8, 3, -4, 6, 13, 21]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1)
t_u = torch.tensor(t_u).unsqueeze(1)
t_u.shape

torch.Size([11, 1])

In [9]:
linear_model = nn.Linear(1, 1)
optimizer = torch.optim.SGD(linear_model.parameters(), lr = 1e-2)

In [10]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[0.2756]], requires_grad=True),
 Parameter containing:
 tensor([-0.1248], requires_grad=True)]

In [11]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs+1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        if epoch == 1 or epoch % 1000 == 0:
            print(f'Epoch{epoch}, Training loss {loss_train.item():.4f},'
                 f'Validation loss{loss_val.item():.4f}')

In [12]:
train_t_un = train_t_un.unsqueeze(1)
val_t_un = val_t_un.unsqueeze(1)
train_t_c = train_t_c.unsqueeze(1)
val_t_c = val_t_c.unsqueeze(1)

In [13]:
linear_model = nn.Linear(1, 1)
optimizer = torch.optim.SGD(linear_model.parameters(), lr = 1e-2)
training_loop(n_epochs = 3000, optimizer = optimizer, model = linear_model, loss_fn = nn.MSELoss(), t_u_train = train_t_un, t_u_val = val_t_un, t_c_train = train_t_c, t_c_val = val_t_c)

Epoch1, Training loss 60.9637,Validation loss243.3638
Epoch1000, Training loss 5.8775,Validation loss21.0489
Epoch2000, Training loss 3.5764,Validation loss5.9003
Epoch3000, Training loss 3.1020,Validation loss3.3834


In [14]:
val_t_c

tensor([[-4.],
        [28.]])

In [17]:
seq_model = nn.Sequential(nn.Linear(1,13),
                         nn.Tanh(),
                          nn.Linear(13,1))

In [18]:
seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [23]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

0.weight torch.Size([13, 1])
0.bias torch.Size([13])
2.weight torch.Size([1, 13])
2.bias torch.Size([1])


In [28]:
optimizer = torch.optim.SGD(seq_model.parameters(), lr = 1e-03)
training_loop(n_epochs = 5000,
             optimizer = optimizer,
             model = seq_model,
              loss_fn = nn.MSELoss(),
             t_u_train = train_t_un,
             t_u_val = val_t_un,
             t_c_train = train_t_c,
             t_c_val = val_t_c)

Epoch1, Training loss 144.4401,Validation loss406.0864
Epoch1000, Training loss 7.3061,Validation loss60.3449
Epoch2000, Training loss 3.2509,Validation loss20.4819
Epoch3000, Training loss 2.7021,Validation loss12.0027
Epoch4000, Training loss 3.9093,Validation loss11.7916
Epoch5000, Training loss 2.6067,Validation loss8.0332


In [30]:
print('output: ', seq_model(val_t_un))

output:  tensor([[-2.1294],
        [25.1204]], grad_fn=<AddmmBackward0>)


In [31]:
print('answer: ', val_t_c)

answer:  tensor([[-4.],
        [28.]])


In [32]:
print('hidden: ', seq_model.hidden_linear.weight.grad)

AttributeError: 'Sequential' object has no attribute 'hidden_linear'